In [ ]:
import tqdm as notebook_tqdm
import torch
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Dict, Optional, cast
from torch import Tensor
from collections import OrderedDict 
from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck
from torchvision.models.resnet import ResNet50_Weights
import json
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
le = LabelEncoder()
class WebDataset(Dataset):
    def __init__(self, images, categories, transform=None, target_transform=None):
        self.images = images
        self.categories = le.fit_transform(categories)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], torch.tensor(self.categories[idx])

In [ ]:
f = open('items_shuffle_1000.json')
json_data = json.load(f)
# data_subset = json_data[:5]
dataset = WebDataset(images, categories)
dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

In [ ]:
from torchvision import models
class ResNet101(models.ResNet):
    def __init__(self, num_classes=1000, pretrained=True, **kwargs):
        # Start with the standard resnet101
        super().__init__(
            block=models.resnet.Bottleneck,
            layers=[3, 4, 23, 3],
            num_classes=num_classes,
            **kwargs
        )
        if pretrained:
            state_dict = torch.hub.load_state_dict_from_url(
                "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth",
                progress=True
            )
            self.load_state_dict(state_dict)
 
    # Reimplementing forward pass.
    # Replacing the forward inference defined here 
    # http://tiny.cc/23pmmz
    def _forward_impl(self, x):
        # Standard forward for resnet
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
 
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
 
        # Notice there is no forward pass through the original classifier.
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
 
        return x

In [ ]:
model2 = ResNet101(pretrained=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model2.to(device)
model2.eval()

In [ ]:
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
features = torch.empty((0, 2048), dtype=torch.float32)
batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)
for i, (images, labels) in enumerate(dataloader):
        
        # Move images to device
        images, labels = images.to(device), labels.to(device)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model2(images)
        outputs = outputs.cpu().numpy()
        features = np.concatenate((features, outputs))

In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2).fit_transform(features)

In [ ]:
# scale and move the coordinates so they fit [0; 1] range
def scale_to_01_range(x):
    # compute the distribution range
    value_range = (np.max(x) - np.min(x))
 
    # move the distribution so that it starts from zero
    # by extracting the minimal value from all its values
    starts_from_zero = x - np.min(x)
 
    # make the distribution fit [0; 1] by dividing by its range
    return starts_from_zero / value_range
 
# extract x and y coordinates representing the positions of the images on T-SNE plot
tx = tsne[:, 0]
ty = tsne[:, 1]
 
tx = scale_to_01_range(tx)
ty = scale_to_01_range(ty)

In [ ]:
print(list(le.classes_))

In [ ]:
cmap = matplotlib.cm.get_cmap('Dark2')
print(len(cmap.colors))
# len colors >= num classes

In [ ]:
# initialize a matplotlib plot
fig = plt.figure()
ax = fig.add_subplot(111)

colors = cmap.colors
 
# for every class, we'll add a scatter plot separately
for j, label in enumerate(list(le.classes_)):
    # find the samples of the current class in the data
    indices = [i for i, l in enumerate(le.inverse_transform(dataset.categories)) if l == label]
 
    # extract the coordinates of the points of this class only
    current_tx = np.take(tx, indices)
    current_ty = np.take(ty, indices)
 
    # convert the class color to matplotlib format
    # color = np.array(colors_per_class[label], dtype=np.float) / 255
 
    # add a scatter plot with the corresponding color and label
    ax.scatter(current_tx, current_ty, c=colors[j], label=label)
 
# build a legend using the labels we set previously
ax.legend(loc='best')
 
# finally, show the plot
plt.show()